## Anime Recommendation System

DARBALI OUSSAMA
CHAGOUTI EL HOUCINE

## Content Based Filtering

In [ ]:
import pandas as pd
data=pd.read_csv('anime_planet.csv')
data['tags']=data['Title']+data['overview']+data['genres']
data['tags']=data['tags'].str.strip()
data['tags'][0]

'Attack on Titan The Final Season: The Final ChaptersFinal part ofAttack on Titan The Final Season. , Action , Drama , Fantasy , Shounen , Dark Fantasy , Military , War , Based on a Manga'

In [ ]:
data['title']=data['Title']
new_data=data.drop(columns=['overview','genres','Title'])
new_data=new_data[['title','tags','Poster']]
new_data.head()

,title,tags,Poster
0,Attack on Titan The Final Season: The Final Ch...,Attack on Titan The Final Season: The Final Ch...,https://cdn.anime-planet.com/anime/primary/att...
1,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: BrotherhoodThe foundation...,https://cdn.anime-planet.com/anime/primary/ful...
2,Fruits Basket the Final Season,Fruits Basket the Final SeasonThe final arc of...,https://cdn.anime-planet.com/anime/primary/fru...
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Demon Slayer: Kimetsu no Yaiba - Entertainment...,https://cdn.anime-planet.com/anime/primary/dem...
4,Mo Dao Zu Shi 3,Mo Dao Zu Shi 3The third season ofMo Dao Zu Sh...,https://cdn.anime-planet.com/anime/primary/mo-...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=12248, stop_words='english')
cv


CountVectorizer(max_features=12248, stop_words='english')

In [ ]:
vector=cv.fit_transform(new_data['tags'].values.astype('U')).toarray()
vector.shape

(3500, 12248)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
len(similarity[0])

3500

In [ ]:
new_data[new_data['title']=='Naruto']

,title,tags,Poster
893,Naruto,NarutoNaruto Uzumaki is a young ninja who bear...,https://cdn.anime-planet.com/anime/primary/nar...


In [ ]:
distance = sorted(list(enumerate(similarity[893])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:10]:
    print(new_data.iloc[i[0]].title)

Naruto
Naruto Shippuden Movie 3: The Will of Fire
Naruto Shippuden
Naruto Shippuden Movie 4: The Lost Tower
Naruto Shippuden Movie 2: Bonds
Boruto: Naruto the Movie - The Day Naruto Became Hokage
Naruto Shippuden Movie 5: Blood Prison
Naruto Shippuden OVA: Sage Naruto vs Sasuke
Naruto Shippuden Movie 6: Road to Ninja
Naruto Shippuden: Sunny Side Battle!


In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
new_data['tags'] = new_data['tags'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(new_data['tags'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(3500, 14603)

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map of indices and movie titles
indices = pd.Series(new_data.index, index=new_data['title']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return new_data['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Bleach')

3481                 Bleach: Memories in the Rain
1641                Bleach Movie 3: Fade to Black
1943           Bleach Movie 1: Memories of Nobody
5                 Bleach: Thousand-Year Blood War
723                                    Soul Eater
1866    Bleach Movie 2: The DiamondDust Rebellion
1532                       Full Moon wo Sagashite
1094           Yumeiro Pâtissière SP Professional
1683         Death Note Rewrite 2: L's Successors
540                                 Yu Yu Hakusho
Name: title, dtype: object

In [ ]:
import pickle
pickle.dump(new_data, open('anime.pkl', 'wb'))
pickle.dump(cosine_sim, open('similarity.pkl', 'wb'))
pickle.load(open('anime.pkl', 'rb'))

,title,tags,Poster
0,Attack on Titan The Final Season: The Final Ch...,Attack on Titan The Final Season: The Final Ch...,https://cdn.anime-planet.com/anime/primary/att...
1,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: BrotherhoodThe foundation...,https://cdn.anime-planet.com/anime/primary/ful...
2,Fruits Basket the Final Season,Fruits Basket the Final SeasonThe final arc of...,https://cdn.anime-planet.com/anime/primary/fru...
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Demon Slayer: Kimetsu no Yaiba - Entertainment...,https://cdn.anime-planet.com/anime/primary/dem...
4,Mo Dao Zu Shi 3,Mo Dao Zu Shi 3The third season ofMo Dao Zu Sh...,https://cdn.anime-planet.com/anime/primary/mo-...
...,...,...,...
3495,Doranana x Kirara Magic: Doranana,Doranana x Kirara Magic: DorananaThe offiical ...,https://cdn.anime-planet.com/anime/primary/dor...
3496,Owarimonogatari II Recaps,Owarimonogatari II RecapsNo synopsis yet - che...,https://cdn.anime-planet.com/anime/primary/owa...
3497,Jang Geum Ieui Kkum 2,Jang Geum Ieui Kkum 2No synopsis yet - check b...,https://cdn.anime-planet.com/images/anime/defa...
3498,Sengoku Night Blood: Ongaku Dai Tokushuu,Sengoku Night Blood: Ongaku Dai TokushuuNo syn...,https://cdn.anime-planet.com/anime/primary/sen...


## Collaborative Filtering

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357229 sha256=fe02c949d629da92cc94338795ea1e3aa2787b3ce43e3ac03f9b2ec5790ecac9
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
ratings = pd.read_csv('rating_complete.csv')
ratings.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

anime_data = pd.read_csv('anime.csv')

# Préparer les données pour Surprise
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))
data = Dataset.load_from_df(ratings[['user_id', 'anime_id', 'rating']], reader)

# Séparer les données en train et test
trainset, testset = train_test_split(data, test_size=0.25)

# Construire le modèle SVD
algo = SVD()

# Entraîner le modèle sur les données d'entraînement
algo.fit(trainset)

# Faire des prédictions sur le set de test
predictions = algo.test(testset)

# Évaluer le modèle
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Fonction pour faire des recommandations pour un utilisateur donné
def get_recommendations(user_id, algo, ratings, anime_data, num_recommendations=10):
    # Obtenir tous les films que l'utilisateur n'a pas encore notés
    all_anime_ids = ratings['anime_id'].unique()
    rated_anime_ids = ratings[ratings['user_id'] == user_id]['anime_id']
    unrated_anime_ids = [anime_id for anime_id in all_anime_ids if anime_id not in rated_anime_ids.values]

    # Prédire les notes pour les films non notés
    predictions = [algo.predict(user_id, anime_id) for anime_id in unrated_anime_ids]

    # Trier les prédictions par note estimée
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Retourner les meilleurs films recommandés
    top_predictions = predictions[:num_recommendations]
    top_anime_ids = [pred.iid for pred in top_predictions]

    # Obtenir les titres des films recommandés
    recommended_animes = anime_data[anime_data['MAL_ID'].isin(top_anime_ids)][['MAL_ID', 'Name']]
    return list(recommended_animes['Name'])

# Exemple d'utilisation de la fonction de recommandation
user_id = 137
recommendations = get_recommendations(user_id, algo, ratings, anime_data)
print("Anime recommandés pour l'utilisateur {}:".format(user_id))
print(recommendations)


RMSE: 1.2087
RMSE: 1.208663077584039
Anime recommandés pour l'utilisateur 137:
['Mushishi', 'Ginga Eiyuu Densetsu', 'Gintama', "Gintama'", 'Hunter x Hunter (2011)', "Gintama': Enchousen", 'Mushishi Zoku Shou', 'Gintama°', 'Kizumonogatari III: Reiketsu-hen', '3-gatsu no Lion 2nd Season']


## Recommendation system using BERT

In [ ]:
!pip install transformers scikit-learn torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import pandas as pd
anime_data=pd.read_csv('anime_planet.csv')
anime_data=anime_data[['Title','genres','overview']]
# Remplacer les valeurs manquantes par des chaînes vides
anime_data['Title'] = anime_data['Title'].fillna('')
anime_data['overview'] = anime_data['overview'].fillna('')
anime_data['genres'] = anime_data['genres'].fillna('')

# S'assurer que toutes les valeurs sont des chaînes de caractères
anime_data['Title'] = anime_data['Title'].astype(str)
anime_data['overview'] = anime_data['overview'].astype(str)
anime_data['genres'] = anime_data['genres'].astype(str)
anime_data

,Title,genres,overview
0,Attack on Titan The Final Season: The Final Ch...,", Action , Drama , Fantasy , Shounen , Dark F...",Final part ofAttack on Titan The Final Season.
1,Fullmetal Alchemist: Brotherhood,", Action , Adventure , Drama , Fantasy , Myst...",The foundation of alchemy is based on the law ...
2,Fruits Basket the Final Season,", Drama , Fantasy , Romance , Shoujo , Animal...",The final arc ofFruits Basket.
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,", Action , Adventure , Shounen , Demons , His...",Tanjiro and his friends accompany the Hashira ...
4,Mo Dao Zu Shi 3,", Fantasy , Ancient China , Chinese Animation...",The third season ofMo Dao Zu Shi.
...,...,...,...
3495,Doranana x Kirara Magic: Doranana,", Sci Fi , Animal Characteristics","The offiical music video forDoranana, byDorana..."
3496,Owarimonogatari II Recaps,", Recap , Based on a Light Novel",No synopsis yet - check back soon!
3497,Jang Geum Ieui Kkum 2,", Food and Beverage , Korean Animation",No synopsis yet - check back soon!
3498,Sengoku Night Blood: Ongaku Dai Tokushuu,", Fantasy , Non-Human Protagonists , Based on...",No synopsis yet - check back soon!


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Charger le tokenizer et le modèle BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Utiliser la sortie du [CLS] token comme vecteur de texte
    cls_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return cls_embedding

# Encoder les titres, descriptions et genres
anime_data['encoded'] = anime_data.apply(lambda x: encode_text(x['Title'] + ' ' + x['genres'] + ' ' + x['overview']), axis=1)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Fonction pour obtenir des recommandations basées sur la similarité cosinus
def get_recommendations(title, anime_data, top_k=5):
    # Obtenir l'index de l'anime
    idx = anime_data[anime_data['Title'] == title].index[0]
    # Calculer la similarité cosinus entre l'anime et tous les autres animes
    cosine_sim = cosine_similarity(anime_data['encoded'].tolist()[idx], np.vstack(anime_data['encoded'].tolist()))
    # Obtenir les indices des animes les plus similaires
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k + 1]  # Exclure l'anime lui-même
    # Obtenir les titres des animes les plus similaires
    anime_indices = [i[0] for i in sim_scores]
    return anime_data['Title'].iloc[anime_indices]

# Exemple d'utilisation de la fonction de recommandation
title = 'Naruto'
recommendations = get_recommendations(title, anime_data)
print("Animes recommandés pour '{}': {}".format(title, recommendations.tolist()))


Animes recommandés pour 'Naruto': ['Naruto Shippuden Movie 5: Blood Prison', 'Naruto Shippuden Movie 3: The Will of Fire', 'Bleach', 'Tokyo Ravens', 'Guilty Crown']


In [ ]:
import pickle

# Sauvegarder les embeddings encodés dans un fichier
with open("encoded_embeddings.pkl", "wb") as f:
    pickle.dump(anime_data['encoded'].tolist(), f)
